In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!cp -r "/content/gdrive/MyDrive/mmsd_raw_data/utterances_final" "/content/videos"

In [ ]:
%%shell
#!/usr/bin/env bash

set -e

# SCRIPT_DIR="$( cd "$( dirname "${BASH_SOURCE[0]}" )" >/dev/null && pwd )"
# pushd "${SCRIPT_DIR}/.." > /dev/null

videos_folder_path=/content/videos
audios_folder_path=/content/audios
ext=mp4

mkdir -p "${audios_folder_path}"

for video_file_path in "${videos_folder_path}"/*."${ext}"; do
    slash_and_video_file_name="${video_file_path:${#videos_folder_path}}"
    slash_and_video_file_name_without_extension="${slash_and_video_file_name%.${ext}}"
    audio_file_path="${audios_folder_path}${slash_and_video_file_name_without_extension}.aac"
    ffmpeg -i "${video_file_path}" -vn -acodec copy "${audio_file_path}"
done

popd > /dev/null

Streaming output truncated to the last 5000 lines.
      handler_name    : SoundHandler
    Stream #0:2(eng): Data: bin_data (text / 0x74786574), 0 kb/s
    Metadata:
      handler_name    : SubtitleHandler
Output #0, adts, to '/content/audios/2_504.aac':
  Metadata:
    major_brand     : isom
    minor_version   : 512
    compatible_brands: isomiso2avc1mp41
    encoder         : Lavf57.83.100
    Chapter #0:0: start 0.000000, end 2.330000
    Metadata:
      title           : 00:02:12.257
    Stream #0:0(eng): Audio: aac (LC) (mp4a / 0x6134706D), 48000 Hz, 5.1, fltp, 341 kb/s (default)
    Metadata:
      handler_name    : SoundHandler
Stream mapping:
  Stream #0:1 -> #0:0 (copy)
Press [q] to stop, [?] for help
size=      99kB time=00:00:02.32 bitrate= 348.0kbits/s speed=2.86e+03x    
video:0kB audio:98kB subtitle:0kB other streams:0kB global headers:0kB muxing overhead: 0.774167%
ffmpeg version 3.4.11-0ubuntu0.1 Copyright (c) 2000-2022 the FFmpeg developers
  built with gcc 7 (Ubuntu

CalledProcessError: ignored

In [ ]:
import os

_, _, files = next(os.walk("/content/videos"))
file_count = len(files)
video_count=file_count
_, _, files = next(os.walk("/content/audios"))
file_count = len(files)
audio_count=file_count
print(video_count, audio_count)

690 690


In [ ]:
!cp -r "/content/audios" "/content/gdrive/MyDrive/mmsd_raw_data/audio/utterances_final" 

In [ ]:

import os
import pickle
import warnings
warnings.filterwarnings("ignore")
import librosa
import numpy as np
from tqdm.auto import tqdm

AUDIOS_FOLDER = "/content/gdrive/MyDrive/mmsd_raw_data/audio/utterances_final/audios"
AUDIO_FEATURES_PATH = "/content/gdrive/MyDrive/Pickle_Data/audio_features.pkl"


def get_librosa_features(path: str) -> np.ndarray:
    y, sr = librosa.load(path)

    hop_length = 512  # Set the hop length; at 22050 Hz, 512 samples ~= 23ms

    # Remove vocals first
    D = librosa.stft(y, hop_length=hop_length)
    S_full, phase = librosa.magphase(D)

    S_filter = librosa.decompose.nn_filter(S_full, aggregate=np.median, metric="cosine",
                                           width=int(librosa.time_to_frames(0.2, sr=sr)))

    S_filter = np.minimum(S_full, S_filter)

    margin_i, margin_v = 2, 4
    power = 2
    mask_v = librosa.util.softmask(S_full - S_filter, margin_v * S_filter, power=power)
    S_foreground = mask_v * S_full

    # Recreate vocal_removal y
    new_D = S_foreground * phase
    y = librosa.istft(new_D)

    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)  # Compute MFCC features from the raw signal
    mfcc_delta = librosa.feature.delta(mfcc)  # And the first-order differences (delta features)

    S = librosa.feature.melspectrogram(y=y, sr=sr, n_mels=128, fmax=8000)
    S_delta = librosa.feature.delta(S)

    spectral_centroid = librosa.feature.spectral_centroid(S=S_full)

    audio_feature = np.vstack((mfcc, mfcc_delta, S, S_delta, spectral_centroid))  # combine features

    # binning data
    jump = int(audio_feature.shape[1] / 10)
    return librosa.util.sync(audio_feature, range(1, audio_feature.shape[1], jump))


def save_audio_features() -> None:
    audio_feature = {}
    for filename in tqdm(os.listdir(AUDIOS_FOLDER), desc="Computing the audio features"):
        id_ = filename.rsplit(".", maxsplit=1)[0]
        audio_feature[id_] = get_librosa_features(os.path.join(AUDIOS_FOLDER, filename))
        print(audio_feature[id_].shape)
        audio_feature[id_]=np.mean(audio_feature[id_], axis=1)
        print(audio_feature[id_].shape)
    with open(AUDIO_FEATURES_PATH, "wb") as file:
        pickle.dump(audio_feature, file, protocol=2)
    print(len(audio_feature),audio_feature.shape)

def get_audio_duration() -> None:
    filenames = os.listdir(AUDIOS_FOLDER)
    print(sum(librosa.core.get_duration(filename=os.path.join(AUDIOS_FOLDER, filename))
              for filename in tqdm(filenames, desc="Computing the average duration of the audios")) / len(filenames))


def main() -> None:
    get_audio_duration()
    save_audio_features()
    with open(AUDIO_FEATURES_PATH, "rb") as file:
         pickle.load(file)

if __name__ == "__main__":
    main()


Computing the average duration of the audios:   0%|          | 0/690 [00:00<?, ?it/s]

5.362753623188406


Computing the audio features:   0%|          | 0/690 [00:00<?, ?it/s]

(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 11)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 11)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 11)
(283,)
(283, 11)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 11)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 11)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 11)
(283,)
(283, 11)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 11)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 11)
(283,)
(283, 12)
(283,)
(283, 11)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 11)
(283,)
(283, 12)
(283,)
(283, 12)
(283,)
(283, 12)
(283

AttributeError: ignored

In [ ]:
import pickle
import numpy as np
audio_feats = []
with (open("/content/gdrive/MyDrive/Pickle_Data/audio_features.pkl", "rb")) as openfile:
    while True:
        try:
            print("hi")
            audio_feats.append(pickle.load(openfile))
        except EOFError:
            break
audio_feats = np.array(audio_feats)
print(audio_feats)